<a href="https://colab.research.google.com/github/amirhoseinaghaei/Research_Simulation/blob/main/Research_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment

AoI Scheduler

In [ ]:


import numpy as np
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.1
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

Lesson_buffer = LessonBuffer(1000, 25, 5)
episode = 0
rudder_lstm = LSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer, n_units=n_lstm,
                        lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                        lstm_batch_size=8, continuous_pred_factor=0.5)

# rudder_lstm.load_state_dict(torch.load('rudder_lstm.pt'))
environment = Environment(100,25)
environment.CreateStates()
episode = 0
for i in range(200000):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    while not done:
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        elif environment.state.Ra == 1 and environment.state.U == 0:
            action = 1 if  random.uniform(0, 1) < 0 else 0
        elif environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        elif environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 
            # for i in states:
            #     print(i)
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            # print(states)
            # Time.sleep(5)
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            Lesson_buffer.add(states = states, actions = actions, rewards = rewards)
            # print("++++++++++++++++++++++++++")    
            # and Lesson_buffer.different_returns_encountered()    
            if  Lesson_buffer.full_enough() :
                    # print("different_returns_encountered")        
                    # If RUDDER is run, the LSTM is trained after each episode until its loss is below a threshold.
                    # Samples will be drawn from the lessons buffer.
                    if episode % 25 == 0:

                        print("Truee")
                        print(episode)
                        rudder_lstm.train(episode=episode)
                    if episode >= 1600: 
                        torch.save(rudder_lstm.state_dict(), 'rudder_lstm.pt')
                    # Then the LSTM is used to redistribute the reward.
            # print(rewards)
            # print(states)
            # print(actions)
            rewards = rudder_lstm.redistribute_reward(states=np.expand_dims(states, 0),
                                                    actions=np.expand_dims(actions, 0))[0, :]
            # print(rewards)
            # Time.sleep(1)

